In [16]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import h,c,k

In [17]:
hdu2=fits.open('/home/soumadip/Downloads/FIRAS_DESTRIPED_SKY_SPECTRA_LOWF.FITS')
hdu2.info()

Filename: /home/soumadip/Downloads/FIRAS_DESTRIPED_SKY_SPECTRA_LOWF.FITS
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      63   ()      
  1                1 BinTableHDU     44   6067R x 6C   [1J, 1E, 1E, 1E, 1I, 182E]   


#### Contents of the dataset

In [18]:
destriped_data=hdu2[1].data
destriped_data.columns

ColDefs(
    name = 'PIXEL'; format = '1J'
    name = 'GAL_LON'; format = '1E'; unit = 'deg'
    name = 'GAL_LAT'; format = '1E'; unit = 'deg'
    name = 'WEIGHT'; format = '1E'
    name = 'STR_USED'; format = '1I'
    name = 'SPECTRUM'; format = '182E'; unit = 'MJy/sr'; dim = '(182)'
)

#### Getting best fit temperatures for every pixel

It is observed that all the pixels only have 43 intensity values. This makes our job much easier

In [ ]:
c_cms=100*c
from scipy.optimize import curve_fit
def Intensity(wave_num,T):
    return 2*np.pi*h*(wave_num)**3*c_cms**2/(np.exp(h*c_cms*(wave_num)/(k*T))-1)*1e13
    '''
    h = planck constant
    k = boltzmann constant
    c = speed of light in vacuum
    T = temperature of blackbody
    wave_num = wave number of the radiation (1/wavelength)
    '''

In [24]:
# 1st frequency is 68.020812 Ghz
freq_0=68.020812*1e9
freq_interval=13.604162*1e9

#wave number is freq/c. there are 43 total values measured with this interval

wave_number=[]
for i in range(43):
    freq_flag=freq_0+i*freq_interval
    wave_number.append(freq_flag/c_cms)
wave_number=np.array(wave_number)

43

In [25]:

def get_Temperature(wave_number,input):
    Pixel_data=dict()
    Pixel_data['Temperature']=[]
    Pixel_data['Uncertainty']=[]
    for i in range(len(input)):
        Intensity_observed=input[i]
        Intensity_observed=Intensity_observed[0:43:]
        T0=1
        popt,pcov=curve_fit(Intensity,wave_number,Intensity_observed,p0=[T0])
        Temperature_fit=popt[0]
        Uncertainty_fit=np.sqrt(np.diag(pcov))
        Pixel_data['Temperature'].append(Temperature_fit)
        Pixel_data['Uncertainty'].append(Uncertainty_fit)
    return Pixel_data


In [26]:
Pixel_data=get_Temperature(wave_number,destriped_data['SPECTRUM'])


In [31]:
avg_temperature=np.sum(Pixel_data['Temperature'])/len(Pixel_data['Temperature'])
avg_uncertainty=np.sum(Pixel_data['Uncertainty'])/len(Pixel_data['Uncertainty'])
print('T = {:.3f} + {:.3f} K'.format(avg_temperature,avg_uncertainty))

T = 2.772 + 0.003 K


This is far from the given value . We need to remove the galactic contamination it seems


In [37]:
lattitude = destriped_data['GAL_LAT']
Intensity_observed = destriped_data['SPECTRUM']

i=[]
accepted_spectrum=[]

for i in range(0,len(lattitude)):
    if -10 < lattitude[i] < 10:
        continue
    else:
        accepted_spectrum.append(Intensity_observed[i])
spectrum_cut=np.array(accepted_spectrum)
spectrum_cut.shape

(5013, 182)

In [35]:
def get_temp(wave_number,input):
    T=[]
    for i in range(len(input)):
        Intensity_observed=input[i]
        Intensity_observed=Intensity_observed[0:43:]
        T0=2
        popt,pcov=curve_fit(Intensity,wave_number,Intensity_observed,T0)
        k=popt[0]
        T.append(k)
    return T

temperature_cut = get_temp(wave_number,spectrum_cut)
sum(temperature_cut)/len(temperature_cut)

2.770233291707014